In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [2]:
#set global net_buffer_length=1000000; 
#set global max_allowed_packet=1000000000;

In [3]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
contours_edges_table_name="contours_edges"
2

2

In [4]:
DELETE_TABLES=True
if DELETE_TABLES:
    cmd="""DROP TABLE IF EXISTS `%s`"""%contours_edges_table_name
    cursor.execute(cmd)
    cmd="""DROP TABLE IF EXISTS `%s`"""%contours_lines_table_name
    cursor.execute(cmd)
2

2

In [5]:
CREATE_TABLES=True
if CREATE_TABLES:
    cmd="""CREATE TABLE `%s`(
          `id` INT NOT NULL AUTO_INCREMENT,
          `source` INT,
          `is_closed` BOOLEAN NOT NULL,
          `elevation` FLOAT NOT NULL,
          `length` FLOAT,
          `number_points` FLOAT,
          `geometry` LINESTRING NOT NULL,
          PRIMARY KEY (id)
          )
          ENGINE=InnoDB;
        """%contours_lines_table_name
    cursor.execute(cmd)
    
    cmd="""CREATE TABLE `%s`(
    `begin` INT NOT NULL,
    `end` INT NOT NULL,
    FOREIGN KEY (begin) REFERENCES %s(id),
    FOREIGN KEY (end) REFERENCES %s(id) ,
    PRIMARY KEY (begin,end)
    );
        """%(contours_edges_table_name,contours_lines_table_name,contours_lines_table_name)
    
    cursor.execute(cmd)

In [6]:
#ASSUME GEODATAFRAMES ARE IN UTM COORDINATES
GENERATE=True
if GENERATE:
    t1=time.time()
    file_paths=glob.glob('COURBE_1-0__SHP_LAMB93_D02B_2021-01-01/COURBE/1_DONNEES_LIVRAISON_2021-01-01/COURBE_1-0_SHP_LAMB93_D02B_2021/*.shp')
    #file_paths+=glob.glob('COURBE_1-0__SHP_LAMB93_D02A_2021-01-01/COURBE/1_DONNEES_LIVRAISON_2021-01-01/COURBE_1-0_SHP_LAMB93_D02A_2021/*.shp')

    add_contour_lines_to_database(file_paths,cursor,contours_lines_table_name=contours_lines_table_name)


        


    t2=time.time()
    print(t2-t1)
2

COURBE_1-0__SHP_LAMB93_D02B_2021-01-01/COURBE/1_DONNEES_LIVRAISON_2021-01-01/COURBE_1-0_SHP_LAMB93_D02B_2021/COURBE_1200_6200.shp
getting pairs took 0.431502
comparing pairs took 0.000087
COURBE_1-0__SHP_LAMB93_D02B_2021-01-01/COURBE/1_DONNEES_LIVRAISON_2021-01-01/COURBE_1-0_SHP_LAMB93_D02B_2021/COURBE_1160_6160.shp
getting pairs took 3.806966
comparing pairs took 0.005970
COURBE_1-0__SHP_LAMB93_D02B_2021-01-01/COURBE/1_DONNEES_LIVRAISON_2021-01-01/COURBE_1-0_SHP_LAMB93_D02B_2021/COURBE_1160_6200.shp
getting pairs took 3.865918
comparing pairs took 0.000087
COURBE_1-0__SHP_LAMB93_D02B_2021-01-01/COURBE/1_DONNEES_LIVRAISON_2021-01-01/COURBE_1-0_SHP_LAMB93_D02B_2021/COURBE_1200_6240.shp
getting pairs took 2.942117
comparing pairs took 0.000055
COURBE_1-0__SHP_LAMB93_D02B_2021-01-01/COURBE/1_DONNEES_LIVRAISON_2021-01-01/COURBE_1-0_SHP_LAMB93_D02B_2021/COURBE_1200_6120.shp
getting pairs took 7.200703
comparing pairs took 0.000062
COURBE_1-0__SHP_LAMB93_D02B_2021-01-01/COURBE/1_DONNEES_LIVR

2

In [7]:
MERGE=True
if MERGE:
    t1=time.time()
    add_merged_contours_lines(cursor,contours_lines_table_name=contours_lines_table_name)

    t2=time.time()
    print(t2-t1)
2

0.0
5.0
10.0
15.0
20.0
25.0
30.0
35.0
40.0
45.0
50.0
55.0
60.0
65.0
70.0
75.0
80.0
85.0
90.0
95.0
100.0
105.0
110.0
115.0
120.0
125.0
130.0
135.0
140.0
145.0
150.0
155.0
160.0
165.0
170.0
175.0
180.0
185.0
190.0
195.0
200.0
205.0
210.0
215.0
220.0
225.0
230.0
235.0
240.0
245.0
250.0
255.0
260.0
265.0
270.0
275.0
280.0
285.0
290.0
295.0
300.0
305.0
310.0
315.0
320.0
325.0
330.0
335.0
340.0
345.0
350.0
355.0
360.0
365.0
370.0
375.0
380.0
385.0
390.0
395.0
400.0
405.0
410.0
415.0
420.0
425.0
430.0
435.0
440.0
445.0
450.0
455.0
460.0
465.0
470.0
475.0
480.0
485.0
490.0
495.0
500.0
505.0
510.0
515.0
520.0
525.0
530.0
535.0
540.0
545.0
550.0
555.0
560.0
565.0
570.0
575.0
580.0
585.0
590.0
595.0
600.0
605.0
610.0
615.0
620.0
625.0
630.0
635.0
640.0
645.0
650.0
655.0
660.0
665.0
670.0
675.0
680.0
685.0
690.0
695.0
700.0
705.0
710.0
715.0
720.0
725.0
730.0
735.0
740.0
745.0
750.0
755.0
760.0
765.0
770.0
775.0
780.0
785.0
790.0
795.0
800.0
805.0
810.0
815.0
820.0
825.0
830.0
835.0
840.0
845.0
85

2

In [ ]:
COMPUTE_TREE=False

if COMPUTE_TREE:
    t1=time.time()
    edges=[]
    for k,level in enumerate(elevations):
        print(level)
        nearby_levels=[level]
        if k!=0:
            nearby_levels.append(elevations[k-1])
        if k!=len(elevations)-1:
            nearby_levels.append(elevations[k+1])
            
        polys_df=get_level_contours_df(cursor,level,contours_lines_table_name,is_closed=True)
        if polys_df is not None:
            polys_df['geometry']=polys_df['geometry'].apply(lambda ls:Polygon(ls))
            crs=polys_df.estimate_utm_crs()
            polys_df=polys_df.to_crs(crs)
            polys_df['area']=polys_df['geometry'].apply(lambda poly:poly.area)

            points_dfs=[get_level_contours_df(cursor,nearby_level,contours_lines_table_name,is_closed=True) for nearby_level in nearby_levels]
            points_dfs=[df for df in points_dfs if df is not None]
            if len(points_dfs)>0:
                points_df=pd.concat(points_dfs)
                points_df=points_df.to_crs(crs)
                points_df['area']=points_df['geometry'].apply(lambda ls:Polygon(ls).area)
                points_df['geometry']=points_df['geometry'].apply(lambda ls:ls.representative_point())

                inter=gpd.overlay(polys_df,points_df,keep_geom_type=False)
                inter=inter[inter.id_1!=inter.id_2]
                
                edges+=list(zip(inter['id_1'],inter['id_2'],inter['area_1']-inter['area_2']))

    G=nx.DiGraph()
    G.add_edges_from([(u,v,{'weight':area_diff}) for u,v,area_diff in edges])
    for node in G.nodes():
        in_edges=[(u,v,d) for u,v,d in G.in_edges(node,data=True) if u!=v]
        if len(in_edges)>1:
            in_edges=sorted(in_edges,key=lambda edge:edge[2]['weight'])
            for u,v,_ in in_edges[1:]:
                G.remove_edge(u,v)
    tree_edges=G.edges()

    
    string_list=',\n'.join(['(%i,%i)'%(u,v) for (u,v) in tree_edges])
    cmd="""INSERT INTO %s (`begin`,`end`) VALUES %s;"""%(contours_edges_table_name,string_list)
    cursor.execute(cmd)
    
    t2=time.time()
    print(t2-t1)